 Introduction
Cette analyse explore le réseau d'intermédiaires offshore des Paradise Papers via une approche de graphe. Nous identifions les acteurs clés, leurs typologies et les dynamiques transnationales à travers des métriques de centralité et la détection de communautés.

 A. Imports et Configuration Neo4j 

 A.1 Imports des bibliothèques

In [73]:
import pandas as pd
import networkx as nx # Still needed for local graph manipulation and algorithms
import plotly.graph_objects as go
import igraph as ig
import leidenalg
import numpy as np
from collections import Counter
from neo4j import GraphDatabase, basic_auth # 👈 NEW: Neo4j Driver

 Configuration des chemins de données 

In [74]:
nodes_path = r"C:\Users\Ashahi\Desktop\Graph\examen\subgraphs_narratives\subgraph_1_intermediaries_nodes.csv"
edges_path = r"C:\Users\Ashahi\Desktop\Graph\examen\subgraphs_narratives\subgraph_1_intermediaries_edges.csv"

 Connexion à la base Neo4j

In [75]:
URI = "bolt://localhost:7687"
USER = "neo4j"
PASSWORD = "555555555" 

driver = GraphDatabase.driver(URI, auth=basic_auth(USER, PASSWORD))
print("Connexion à Neo4j établie.")

Connexion à Neo4j établie.


  B. Ingestion et préparations des données

 Nettoyage et initialisation de la base

 Wrapper générique pour exécuter une requête Cypher

In [76]:
def execute_cypher(tx, query, params=None):
    if params is None:
        result = tx.run(query)
    else:
        result = tx.run(query, params)
    return [record for record in result]

 Nettoyage - optimisation - importation des nœuds et arrêtes

In [77]:
def setup_neo4j(driver, nodes_path, edges_path):
    with driver.session() as session:
        # 1️⃣ Nettoyage complet 
        session.execute_write(execute_cypher, "MATCH (n) DETACH DELETE n")
        print("🧹 Base Neo4j nettoyée.")

        # 2️⃣ Création d’un index sur node_id (optimisation)
        session.execute_write(execute_cypher, "CREATE INDEX node_id_index IF NOT EXISTS FOR (n:Node) ON (n.node_id)")
        print("⚙️ Index créé sur :Node(node_id).")

        # 3️⃣ Importation des Nœuds
        nodes_df = pd.read_csv(nodes_path, low_memory=False).fillna('')
        node_id_col = next((c for c in nodes_df.columns if c.lower() in ("id", "node_id", "node")), nodes_df.columns[0])
        node_params = nodes_df.rename(columns={node_id_col: 'node_id'}).to_dict('records')

        create_nodes_query = """
        UNWIND $props_list AS props
        MERGE (n:Node {node_id: toString(props.node_id)})
        SET n += apoc.map.clean(props, ['node_id'], [""])
        """

        session.execute_write(execute_cypher, create_nodes_query, {"props_list": node_params})
        print(f"✅ {len(nodes_df)} nœuds importés ou mis à jour.")

        # 4️⃣ Importation des Arêtes
        edges_df = pd.read_csv(edges_path, low_memory=False).fillna('')
        src_col = next((c for c in edges_df.columns if c.lower() in ("source", "from", "node_id_start")), edges_df.columns[0])
        tgt_col = next((c for c in edges_df.columns if c.lower() in ("target", "to", "node_id_end")), edges_df.columns[1])
        edge_params = edges_df.rename(columns={src_col: 'source_id', tgt_col: 'target_id'}).to_dict('records')

        create_edges_query = """
        UNWIND $props_list AS props
        MATCH (a:Node {node_id: toString(props.source_id)})
        MATCH (b:Node {node_id: toString(props.target_id)})
        MERGE (a)-[r:RELATED]->(b)
        SET r += apoc.map.clean(props, ['source_id', 'target_id'], [""])
        """

        session.execute_write(execute_cypher, create_edges_query, {"props_list": edge_params})
        print(f"✅ {len(edges_df)} arêtes importées ou mises à jour.")

    print("Ingestion terminée avec succès.")
    
setup_neo4j(driver, nodes_path, edges_path)

🧹 Base Neo4j nettoyée.
⚙️ Index créé sur :Node(node_id).
✅ 800 nœuds importés ou mis à jour.
✅ 5348 arêtes importées ou mises à jour.
Ingestion terminée avec succès.


 C. Analyse des centralités avec Neo4j GDS

In [78]:
# --------------------------------------------------
# 5️⃣ Mesures de centralité (via Neo4j GDS) – Version robuste
# --------------------------------------------------
def calculate_centrality_gds(driver):
    with driver.session() as session:
        # 0️⃣ Supprimer le graphe projeté s’il existe déjà
        try:
            drop_query = "CALL gds.graph.drop('myGraph', false) YIELD graphName"
            session.execute_write(execute_cypher, drop_query)
            print("Ancien graphe 'myGraph' supprimé.")
        except Exception:
            print("Aucun graphe existant à supprimer (OK).")

        # 1️⃣ Création du graphe projeté
        create_projection_query = """
        CALL gds.graph.project(
            'myGraph',
            'Node',
            'RELATED'
        )
        """
        session.execute_write(execute_cypher, create_projection_query)
        print("✅ Graphe projeté 'myGraph' créé.")

        # 2️⃣ Degree Centrality
        degree_query = """
        CALL gds.degree.write(
            'myGraph',
            {
                writeProperty: 'degree'
            }
        )
        YIELD nodePropertiesWritten
        """
        session.execute_write(execute_cypher, degree_query)
        print("✅ Degree Centrality calculée et écrite.")
        
        # 3️⃣ Betweenness Centrality (sans normalized)
        betw_query = """
        CALL gds.betweenness.write(
            'myGraph',
            {
                writeProperty: 'betweenness',
                concurrency: 4
            }
        )
        YIELD nodePropertiesWritten
        """
        session.execute_write(execute_cypher, betw_query)
        print("✅ Betweenness Centrality calculée et écrite.")

        # 4️⃣ Suppression finale du graphe projeté
        session.execute_write(execute_cypher, "CALL gds.graph.drop('myGraph')")
        print("✅ Graphe projeté supprimé après calculs.")

# Lancer la fonction
calculate_centrality_gds(driver)

Ancien graphe 'myGraph' supprimé.
✅ Graphe projeté 'myGraph' créé.
✅ Degree Centrality calculée et écrite.


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('schema' returned by 'gds.graph.drop' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: "CALL gds.graph.drop('myGraph')"


✅ Betweenness Centrality calculée et écrite.
✅ Graphe projeté supprimé après calculs.


 D. Extraction COMPLÈTE du graphe depuis Neo4j


In [79]:
# --------------------------------------------------
# 6) Extraction de TOUTE la structure du graphe depuis Neo4j
# --------------------------------------------------
def extract_complete_graph_from_neo4j(driver):
    """Extrait la structure complète du graphe + attributs depuis Neo4j"""
    
    # Query pour obtenir tous les nœuds avec leurs attributs
    nodes_query = """
    MATCH (n:Node)
    RETURN 
        toString(n.node_id) AS id,
        n.name AS name,
        n.node_type AS node_type,
        n.jurisdiction_description AS jurisdiction_description,
        n.countries AS countries,
        n.degree AS degree,
        n.betweenness AS betweenness
    """
    
    # Query pour obtenir toutes les arêtes
    edges_query = """
    MATCH (a:Node)-[r:RELATED]->(b:Node)
    RETURN 
        toString(a.node_id) AS source,
        toString(b.node_id) AS target
    """
    
    with driver.session() as session:
        # Extraire les nœuds
        nodes_result = session.run(nodes_query)
        nodes_df = pd.DataFrame([dict(record) for record in nodes_result])
        
        # Extraire les arêtes
        edges_result = session.run(edges_query)
        edges_df = pd.DataFrame([dict(record) for record in edges_result])
    
    return nodes_df, edges_df

# Extraire les données depuis Neo4j
nodes_df, edges_df = extract_complete_graph_from_neo4j(driver)
print(f"✅ Données extraites de Neo4j: {len(nodes_df)} nœuds, {len(edges_df)} arêtes")

✅ Données extraites de Neo4j: 800 nœuds, 5348 arêtes


 E. Reconstruction et enrichissement NetworkX

In [80]:
# --------------------------------------------------
# 7) Construction du graphe NetworkX DEPUIS NEO4J
# --------------------------------------------------

# Construire le graphe NetworkX à partir des données Neo4j
G = nx.from_pandas_edgelist(edges_df, source='source', target='target')

print(f"Graphe NetworkX depuis Neo4j: n={G.number_of_nodes()}, m={G.number_of_edges()}")

# Ajouter tous les attributs des nœuds
print("🔄 Ajout des attributs depuis Neo4j...")
nodes_dict = nodes_df.set_index('id').to_dict('index')

for node_id, attrs in nodes_dict.items():
    if node_id in G.nodes():
        for attr_name, attr_value in attrs.items():
            G.nodes[node_id][attr_name] = attr_value

print(f"✅ Graphe NetworkX enrichi: {len(G.nodes())} nœuds avec attributs")



Graphe NetworkX depuis Neo4j: n=778, m=5348
🔄 Ajout des attributs depuis Neo4j...
✅ Graphe NetworkX enrichi: 778 nœuds avec attributs


In [81]:
# Vérification
sample_node = list(G.nodes())[0]
print(f"📋 Exemple de nœud {sample_node}:")
print(f"   - Type: {G.nodes[sample_node].get('node_type', 'N/A')}")
print(f"   - Degré: {G.nodes[sample_node].get('degree', 'N/A')}")
print(f"   - Betweenness: {G.nodes[sample_node].get('betweenness', 'N/A')}")

📋 Exemple de nœud 81027087:
   - Type: Address
   - Degré: 2.0
   - Betweenness: 0.0


 F. Détection de communautés (Leiden)

In [82]:
# --------------------------------------------------
# 8) Calcul Leiden sur le graphe Neo4j
# --------------------------------------------------
print("🔍 Calcul des communautés Leiden sur le graphe Neo4j...")

# Filtrer les nœuds avec des connexions
nodes_with_edges = [n for n in G.nodes() if G.degree(n) > 0]
G_connected = G.subgraph(nodes_with_edges)

if len(G_connected.nodes()) > 0:
    # Mapping pour igraph
    mapping = {n: i for i, n in enumerate(G_connected.nodes())}
    inv_mapping = {i: n for n, i in mapping.items()}

    # Création du graphe igraph depuis les données Neo4j
    IG = ig.Graph(edges=[(mapping[u], mapping[v]) for u, v in G_connected.edges()])
    
    # Calcul Leiden
    partition = leidenalg.find_partition(IG, leidenalg.ModularityVertexPartition)
    leiden_labels = {inv_mapping[i]: part for i, part in enumerate(partition.membership)}
    
    # Appliquer les labels Leiden
    for node_id, leiden_id in leiden_labels.items():
        if node_id in G.nodes():
            G.nodes[node_id]['leiden'] = leiden_id
    
    print(f"✅ Communautés Leiden calculées: {len(set(leiden_labels.values()))} communautés")
else:
    print("❌ Aucun nœud avec des connexions!")

# Marquer les nœuds sans communauté
for node_id in G.nodes():
    if 'leiden' not in G.nodes[node_id]:
        G.nodes[node_id]['leiden'] = -1

print("🎯 Graphe NetworkX PRÊT pour l'analyse avancée!")

🔍 Calcul des communautés Leiden sur le graphe Neo4j...
✅ Communautés Leiden calculées: 8 communautés
🎯 Graphe NetworkX PRÊT pour l'analyse avancée!


 Diagnostic des données 

In [83]:
# =====================================================
# 1️⃣ VÉRIFICATION DE BASE
# =====================================================
print("\n📊 STATISTIQUES DE BASE:")
print("-" * 25)

print(f"• Nœuds chargés: {len(nodes_df):,}")
print(f"• Relations chargées: {len(edges_df):,}")
print(f"• Graphe NetworkX: {G.number_of_nodes():,} nœuds, {G.number_of_edges():,} arêtes")

# Vérification de la cohérence
print(f"\n✅ VÉRIFICATIONS:")
print(f"• Nœuds avec attributs: {sum(1 for n in G.nodes() if len(G.nodes[n]) > 0):,}")
print(f"• Nœuds sans attributs: {sum(1 for n in G.nodes() if len(G.nodes[n]) == 0):,}")


📊 STATISTIQUES DE BASE:
-------------------------
• Nœuds chargés: 800
• Relations chargées: 5,348
• Graphe NetworkX: 778 nœuds, 5,348 arêtes

✅ VÉRIFICATIONS:
• Nœuds avec attributs: 778
• Nœuds sans attributs: 0


In [84]:
# =====================================================
# 2️⃣ APERÇU DES DONNÉES
# =====================================================
print("\n📋 APERÇU DES PREMIERS NŒUDS:")
print("-" * 30)

# Aperçu des premiers nœuds avec leurs attributs
sample_nodes = list(G.nodes())[:3]
for i, node_id in enumerate(sample_nodes):
    node_data = G.nodes[node_id]
    print(f"\n{i+1}. {node_id}")
    for key, value in list(node_data.items())[:4]:  # Premier 4 attributs
        print(f"   {key}: {str(value)[:50]}{'...' if len(str(value)) > 50 else ''}")


📋 APERÇU DES PREMIERS NŒUDS:
------------------------------

1. 81027087
   name: Jayla Place; Wickhams Cay 1; Road Town; Tortola; B...
   node_type: Address
   jurisdiction_description: None
   countries: British Virgin Islands

2. 80042906
   name: Burns - Michael J
   node_type: Officer
   jurisdiction_description: None
   countries: Bermuda;British Virgin Islands;Canada

3. 80000392
   name: Appleby Corporate Services (BVI) Limited
   node_type: Intermediary
   jurisdiction_description: None
   countries: British Virgin Islands


In [85]:
# =====================================================
# 3️⃣ DISTRIBUTION DES TYPES DE NŒUDS
# =====================================================
print("\n🏷️ TYPES DE NŒUDS:")
print("-" * 20)

node_types = []
for n in G.nodes():
    node_type = G.nodes[n].get('node_type', 'Non spécifié')
    node_types.append(node_type)

type_counts = pd.Series(node_types).value_counts()
for typ, count in type_counts.head(10).items():
    print(f"• {typ}: {count}")


🏷️ TYPES DE NŒUDS:
--------------------
• Entity: 738
• Officer: 31
• Address: 5
• Intermediary: 4


In [86]:
# Visualisation simple
import plotly.express as px

if len(type_counts) > 0:
    fig = px.bar(x=type_counts.index[:8], y=type_counts.values[:8],
                 title="Types de Nœuds dans le Graphe",
                 labels={'x': 'Type', 'y': 'Nombre'})
    fig.show()

In [87]:
# =====================================================
# 4️⃣ DISTRIBUTION GÉOGRAPHIQUE
# =====================================================
print("\n🌍 RÉPARTITION GÉOGRAPHIQUE:")
print("-" * 30)

jurisdictions = []
for n in G.nodes():
    jurisdiction = G.nodes[n].get('jurisdiction_description', 'Non spécifié')
    if jurisdiction and jurisdiction != 'Non spécifié':
        jurisdictions.append(jurisdiction)

jurisdiction_counts = pd.Series(jurisdictions).value_counts().head(10)

for jur, count in jurisdiction_counts.items():
    print(f"• {jur}: {count}")

if len(jurisdiction_counts) > 0:
    fig = px.pie(values=jurisdiction_counts.values, names=jurisdiction_counts.index,
                 title="Top 10 Juridictions")
    fig.show()


🌍 RÉPARTITION GÉOGRAPHIQUE:
------------------------------
• Bermuda: 734
• Jersey: 2
• Cayman Islands: 1
• Turks and Caicos Islands: 1


In [88]:
# =====================================================
# 5️⃣ ANALYSE DE LA CONNECTIVITÉ
# =====================================================
print("\n📈 CONNECTIVITÉ DU RÉSEAU:")
print("-" * 25)

degrees = [d for n, d in G.degree()]
if degrees:
    print(f"• Degré moyen: {np.mean(degrees):.1f}")
    print(f"• Degré max: {max(degrees)}")
    print(f"• Densité du réseau: {nx.density(G):.4f}")

    # Histogramme des degrés
    fig = px.histogram(x=degrees, nbins=50, 
                      title="Distribution du Nombre de Connexions",
                      labels={'x': 'Nombre de connexions', 'y': 'Nombre de nœuds'})
    fig.show()


📈 CONNECTIVITÉ DU RÉSEAU:
-------------------------
• Degré moyen: 13.7
• Degré max: 744
• Densité du réseau: 0.0177


In [89]:
# %% --------------------------------------------------
# 📊 CALCULS ET SIGNIFICATIONS DES TYPOLOGIES
# --------------------------------------------------



Avant la visualisation et les calculs, on va voir certaines notions et définitions pour pouvoir construire, comprendre, et interpréter nos futures visualisations

 Typologies d’acteurs dans les réseaux offshores

Les réseaux issus des Panama Papers révèlent différents rôles selon la position et l’influence des acteurs dans le graphe.  
Ces rôles sont déduits automatiquement à partir de la centralité (betweenness) et du degré de connexion.


 ARCHITECTE  
→ Le cerveau du système  
Cabinets d’avocats ou sociétés fiduciaires qui créent et contrôlent les structures offshores.  
- Très haute centralité et degré  
- Position stratégique : tout passe par eux  


 PASSERELLE  
→ Le pont entre les groupes  
Relie les architectes, les clients et les sociétés-écrans dans plusieurs juridictions.  
- Centralité moyenne à élevée  
- Peu de liens, mais très stratégiques  


 SATELLITE  
→ L’actif sans pouvoir  
Souvent un dirigeant nominal ou officier administratif présent sur plusieurs entités.  
- Beaucoup de connexions (haut degré)  
- Faible centralité, peu d’influence  


 INTERMÉDIAIRE SECONDAIRE  
→ Le suiveur du système  
Petits acteurs locaux (comptables, agents) qui exécutent les instructions.  
- Centralité et degré moyens  
- Rôle d’appui, sans impact stratégique  


 ISOLÉ  
→ Le solitaire  
Entités marginales ou sociétés inactives, peu connectées au reste du réseau.  
- Très faible degré  
- Centralité quasi nulle  


 En résumé
- Architecte = le cerveau  
- Passerelle = le lien  
- Satellite = le visage visible  
- Intermédiaire = le soutien  
- Isolé = l’écarté  

Ces catégories permettent de lire la structure cachée du pouvoir financier dans les réseaux offshores.


In [90]:
# %% --------------------------------------------------
# 🌐 UNIVERS 1 – INTERMÉDIAIRES CENTRAUX : ANALYSE AVANCÉE (Top 35)
# Version optimisée - Hover lisible et légende simplifiée
# --------------------------------------------------
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
from collections import Counter
import numpy as np

# =====================================================
# 1️⃣ CONFIGURATION ET PARAMÈTRES VISUELS
# =====================================================
COLOR_PALETTE = {
    'background': '#f8f9fa',
    'paper': '#ffffff',
    'grid': '#e9ecef',
    'text': '#2c3e50',
    'accent': '#3498db'
}

TYPO_COLORS = {
    "Architecte": "#e74c3c",
    "Passerelle": "#f39c12",
    "Satellite": "#27ae60",
    "Intermédiaire secondaire": "#95a5a6",
    "Isolé": "#bdc3c7",
    "Inconnu": "#7f8c8d"
}

SYMBOL_MAP = {
    "Company": "circle",
    "Officer": "square",
    "Address": "triangle-up",
    "Entity": "diamond",
    "N/A": "cross"
}

In [91]:
# =====================================================
# 2️⃣ EXTRACTION ET ANALYSE DES DONNÉES
# =====================================================
attrs = ["name", "node_type", "jurisdiction_description", "countries", "degree", "betweenness", "leiden"]

print("🔍 Extraction des données en cours...")
data = []
for n, d in G.nodes(data=True):
    row = {"id": n}
    for a in attrs:
        row[a] = d.get(a)
    data.append(row)

df = pd.DataFrame(data).sort_values("betweenness", ascending=False).reset_index(drop=True)

# Nettoyage et enrichissement des données
df["jurisdiction_description"] = df.apply(
    lambda row: row["jurisdiction_description"]
                if pd.notna(row["jurisdiction_description"]) and row["jurisdiction_description"] != ""
                else row["countries"],
    axis=1
)

# Classification avancée
def classify_advanced(row):
    deg, bet = row["degree"], row["betweenness"]
    if pd.isna(deg) or pd.isna(bet):
        return "Inconnu"

    bet_threshold_high = df["betweenness"].quantile(0.95)
    bet_threshold_medium = df["betweenness"].quantile(0.85)
    deg_threshold_high = df["degree"].quantile(0.90)

    if bet > bet_threshold_high and deg > deg_threshold_high:
        return "Architecte"
    elif bet > bet_threshold_medium:
        return "Passerelle"
    elif deg > deg_threshold_high:
        return "Satellite"
    elif deg < df["degree"].quantile(0.25):
        return "Isolé"
    else:
        return "Intermédiaire secondaire"

df["typologie"] = df.apply(classify_advanced, axis=1)

# Communautés
leiden_countries = (
    df.groupby("leiden")["jurisdiction_description"]
    .apply(lambda x: Counter([v for v in x if pd.notna(v)]).most_common(1)[0][0]
           if any(pd.notna(x)) else "Non spécifié")
    .to_dict()
)
df["pays_dominant_communauté"] = df["leiden"].map(leiden_countries)

🔍 Extraction des données en cours...


In [92]:
# Top 35
top35 = df.head(35).copy()

In [93]:
# =====================================================
# 3️⃣ PRÉPARATION DU GRAPHE
# =====================================================
top_ids = set(top35["id"])
G_focus = G.subgraph(top_ids).copy()

print("🔄 Calcul du layout...")
pos = nx.spring_layout(G_focus, k=2, iterations=50, seed=42)

🔄 Calcul du layout...


In [94]:
# =====================================================
# 4️⃣ CRÉATION DE LA VISUALISATION AVANCÉE
# =====================================================


# Arêtes
edge_x, edge_y = [], []
for edge in G_focus.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.8, color='rgba(169, 169, 169, 0.4)'),
    hoverinfo='none',
    mode='lines',
    showlegend=False
)

In [95]:
# Nœuds par type
node_traces = []
node_types = sorted(top35["node_type"].dropna().unique())

for node_type in node_types:
    subset = top35[top35["node_type"] == node_type]

    node_x, node_y, sizes, colors, borders, hover_texts = [], [], [], [], [], []

    for _, row in subset.iterrows():
        node_id = row["id"]
        if node_id not in pos:
            continue

        x, y = pos[node_id]
        node_x.append(x)
        node_y.append(y)

        size = 15 + np.log1p(row["degree"]) * 8
        sizes.append(size)

        colors.append(row["leiden"])
        borders.append(TYPO_COLORS.get(row["typologie"], "#7f8c8d"))

        hover_text = (
            f"<span style='color:white;'>"
            f"<b>{row['name']}</b><br>"
            f"<b>Type:</b> {row['node_type']} | <b>Typologie:</b> {row['typologie']}<br>"
            f"<b>Pays:</b> {row['jurisdiction_description']}<br>"
            f"<b>Degré:</b> {row['degree']} | <b>Betweenness:</b> {row['betweenness']:.4f}<br>"
            f"<b>Communauté Leiden:</b> {row['leiden']}<br>"
            f"<b>Pays dominant:</b> {row['pays_dominant_communauté']}"
            f"</span>"
        )
        hover_texts.append(hover_text)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        marker=dict(
            symbol=SYMBOL_MAP.get(node_type, "circle"),
            size=sizes,
            color=colors,
            colorscale='Viridis',
            showscale=False,
            opacity=0.9,
            line=dict(width=3, color=borders)
        ),
        text=hover_texts,
        hoverinfo='text',
        name=node_type,
        hovertemplate='%{text}<extra></extra>'
    )
    node_traces.append(node_trace)

In [96]:
# =====================================================
# 5️⃣ MISE EN PAGE OPTIMISÉE AVEC LABELS VISIBLES
# =====================================================
fig = go.Figure(data=[edge_trace] + node_traces)

# AJOUT DES ANNOTATIONS POUR LES NŒUDS PRINCIPAUX
top_10_nodes = top35.nlargest(10, 'betweenness')

annotations = []
for _, row in top_10_nodes.iterrows():
    node_id = row['id']
    if node_id in pos:
        x, y = pos[node_id]
        
        # Créer le texte de l'annotation
        name = row['name']
        short_name = name.split(';')[0] if ';' in name else name
        short_name = short_name[:25] + "..." if len(short_name) > 25 else short_name
        
        annotation_text = (
            f"<b>{short_name}</b><br>"
            f"Pays dominant : {row['pays_dominant_communauté']}<br>"
            f"Betweenness: {row['betweenness']:.0f}<br>"
            f"Degré: {row['degree']}"
        )
        
        annotations.append(
            dict(
                x=x,
                y=y,
                xref="x",
                yref="y",
                text=annotation_text,
                showarrow=True,
                arrowhead=2,
                arrowsize=1,
                arrowwidth=2,
                arrowcolor="#2c3e50",
                bgcolor="rgba(255, 255, 255, 0.95)",
                bordercolor="rgba(0, 0, 0, 0.3)",
                borderwidth=1,
                borderpad=4,
                font=dict(size=9, color="#2c3e50"),
                # Position intelligente de l'annotation
                ax=20 if x < 0 else -20,
                ay=-30 if y < 0 else 30
            )
        )

leiden_values = top35["leiden"].dropna()
if not leiden_values.empty:
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(
            colorscale='Viridis',
            cmin=leiden_values.min(),
            cmax=leiden_values.max(),
            colorbar=dict(
                title=dict(text="<b>Communauté Leiden</b>", font=dict(size=12)),
                thickness=20,
                len=0.4,
                y=0.8,
                yanchor='top',
                x=1.02,
                xanchor='left',
                tickfont=dict(size=10)
            ),
            showscale=True
        ),
        hoverinfo='none',
        showlegend=False
    ))

fig.update_layout(
    title=dict(
        text="🌍 UNIVERS 1 – TOP 35 INTERMÉDIAIRES CENTRAUX<br>"
             "<span style='font-size:13px; color:#666'>"
             "Couleur = Communauté • Bordure = Typologie • Taille = Degré<br>"
             "📌 Labels: Top 10 par Betweenness"
             "</span>",
        x=0.5,
        xanchor="center",
        y=0.97,
        font=dict(size=18, color=COLOR_PALETTE['text'])
    ),
    width=1400,
    height=950,
    showlegend=True,
    hoverlabel=dict(
        bgcolor="rgba(30,30,30,0.9)",
        font_size=12,
        font_family="Arial",
        font_color="white",
        bordercolor="rgba(255,255,255,0.2)"
    ),
    plot_bgcolor=COLOR_PALETTE['background'],
    paper_bgcolor=COLOR_PALETTE['paper'],
    legend=dict(
        title=dict(text="<b>Types de nœuds</b>", font=dict(size=13)),
        bgcolor="rgba(255,255,255,0.95)",
        bordercolor="rgba(0,0,0,0.1)",
        borderwidth=1,
        font=dict(size=11),
        x=1.02,
        y=0.15,
        xanchor='left',
        yanchor='bottom',
        traceorder="normal",
        itemsizing="constant"
    ),
    margin=dict(l=80, r=200, t=120, b=80),
    xaxis=dict(showgrid=True, gridcolor=COLOR_PALETTE['grid'], zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=True, gridcolor=COLOR_PALETTE['grid'], zeroline=False, showticklabels=False),
    # AJOUT DES ANNOTATIONS
    annotations=annotations + [
        dict(
            x=0.02, y=0.98,
            xref="paper", yref="paper",
            text="<b>LÉGENDE</b>",
            showarrow=False,
            font=dict(size=14, color=COLOR_PALETTE['text'], family="Arial Black"),
            xanchor="left", yanchor="top",
            bgcolor="rgba(255,255,255,0.9)",
            bordercolor="rgba(0,0,0,0.2)",
            borderwidth=1, borderpad=8
        ),
        dict(
            x=0.02, y=0.93,
            xref="paper", yref="paper",
            text="<b>Typologies:</b>",
            showarrow=False,
            font=dict(size=12, color=COLOR_PALETTE['text']),
            xanchor="left", yanchor="top"
        )
    ]
)

# Ajouter les typologies à la légende
y_pos = 0.88
for typo, color in TYPO_COLORS.items():
    fig.add_annotation(
        x=0.02, y=y_pos,
        xref="paper", yref="paper",
        text=f"<span style='color:{color}'>■</span> {typo}",
        showarrow=False,
        font=dict(size=10, color=COLOR_PALETTE['text']),
        xanchor="left", yanchor="top"
    )
    y_pos -= 0.04

# Note sur les labels
fig.add_annotation(
    x=0.02, y=0.02,
    xref="paper", yref="paper",
    text="<span style='font-size:10px; color:#666'>Les positions représentent la structure topologique du réseau<br>📌 Labels visibles: Top 10 nœuds par Betweenness</span>",
    showarrow=False,
    font=dict(size=9, color=COLOR_PALETTE['text']),
    xanchor="left", yanchor="bottom"
)

print("🚀 Affichage du graphique avec labels visibles...")
print("📌 Les 10 nœuds les plus centraux sont annotés directement")
fig.show()

🚀 Affichage du graphique avec labels visibles...
📌 Les 10 nœuds les plus centraux sont annotés directement


In [97]:
fig.write_html(
    r"C:\Users\Ashahi\Desktop\Graph\examen\notebooks\graphes_notebook1\graphe_1.html",
    include_plotlyjs='cdn',
    full_html=True
)


In [98]:
# =====================================================
# 6️⃣ STATISTIQUES RÉCAPITULATIVES
# =====================================================
print("\n📊 STATISTIQUES DU RÉSEAU ANALYSÉ:")
print(f"   • Nœuds totaux: {len(G_focus)}")
print(f"   • Arêtes totales: {len(G_focus.edges())}")
print(f"   • Densité du réseau: {nx.density(G_focus):.4f}")
print(f"   • Typologies: {dict(Counter(top35['typologie']))}")
print(f"   • Communautés Leiden uniques: {top35['leiden'].nunique()}")


📊 STATISTIQUES DU RÉSEAU ANALYSÉ:
   • Nœuds totaux: 35
   • Arêtes totales: 104
   • Densité du réseau: 0.1748
   • Typologies: {'Architecte': 22, 'Passerelle': 13}
   • Communautés Leiden uniques: 6


In [99]:
print(f"\n📈 TOP 35 - RÉPARTITION:")
top35_typo_counts = top35["typologie"].value_counts()
for typo, count in top35_typo_counts.items():
    percentage = (count / len(top35)) * 100
    print(f"  • {typo}: {count} nœuds ({percentage:.1f}%)")


📈 TOP 35 - RÉPARTITION:
  • Architecte: 22 nœuds (62.9%)
  • Passerelle: 13 nœuds (37.1%)


### Interprétation

Le réseau étudié compte 35 nœuds et 104 liens, avec une densité moyenne (0.17). Cela montre des connexions sélectives, concentrées autour d’un petit nombre d’acteurs-clés.

Dans le Top 35, cette tendance s’inverse : 62,9 % sont des architectes et 37,1 % des passerelles, preuve que le pouvoir structurel est concentré dans ce noyau restreint.
Les architectes, comme Appleby Services ou Argyle House, conçoivent et hébergent les structures offshore, tandis que les passerelles assurent les connexions entre communautés.


Le réseau est très centralisé autour des Bermudes, où ces entités agissent comme de véritables hubs reliant des dizaines de sociétés.
Des individus tels que Michael J. Burns ou Timothy J. Counsell jouent le rôle d’intermédiaires humains entre entreprises et adresses offshore.


Les communautés détectées par l’algorithme de Leiden montrent une organisation par juridiction :
- les Bermudes forment le cœur du réseau,
- les Îles Caïmans jouent un rôle périphérique


En somme, le graphe illustre un système offshore organisé et inégalitaire, où quelques architectes façonnent et contrôlent la circulation mondiale des capitaux, renforçant la fracture économique mondiale.

 Architectes + Passerelles

In [100]:
# 1️⃣ Filtrer Architectes et Passerelles (Top 35)
mini_df = top35[top35['typologie'].isin(['Architecte','Passerelle'])].copy()
mini_ids = set(mini_df['id'])
G_mini = G.subgraph(mini_ids).copy()

print(f"Sous-graphe cœur : {len(G_mini.nodes())} nœuds, {len(G_mini.edges())} connexions internes")

# 2️⃣ Layout plus espacé
pos_mini = nx.spring_layout(G_mini, k=3, iterations=100, seed=42)  # k augmenté pour plus d'espace

# 3️⃣ Arêtes discrètes
edge_x, edge_y = [], []
for u,v in G_mini.edges():
    x0, y0 = pos_mini[u]
    x1, y1 = pos_mini[v]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.8, color='rgba(120,120,120,0.3)'),
    hoverinfo='none',
    mode='lines',
    showlegend=False
)

# 4️⃣ Nœuds avec annotations MINIMALISTES
node_traces = []
annotations = []

# Sélectionner seulement les 5-6 nœuds les plus importants pour les annotations
top_annotations = mini_df.nlargest(6, 'betweenness')

for typ in ['Architecte','Passerelle']:
    subset = mini_df[mini_df['typologie']==typ]
    node_x, node_y, sizes, borders, hover_texts = [], [], [], [], []
    
    for _, row in subset.iterrows():
        nid = row['id']
        if nid not in pos_mini:
            continue
        x, y = pos_mini[nid]
        node_x.append(x)
        node_y.append(y)
        
        # Taille basée sur la centralité (plus significative)
        size = 20 + (row['betweenness'] / mini_df['betweenness'].max()) * 40
        sizes.append(size)
        
        borders.append(TYPO_COLORS.get(row['typologie'], "#7f8c8d"))
        hover_texts.append(f"{row['name']} | {row['typologie']}")
    
    node_traces.append(go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        marker=dict(
            symbol='circle', 
            size=sizes, 
            color=borders, 
            line=dict(width=2, color='#000000')
        ),
        text=hover_texts,
        hoverinfo='text',
        name=typ
    ))

# 5️⃣ Annotations seulement pour les TOP 6
for _, row in top_annotations.iterrows():
    nid = row['id']
    if nid in pos_mini:
        x, y = pos_mini[nid]
        
        name = row['name']
        short_name = name.split(';')[0] if ';' in name else name
        short_name = short_name[:12] + "..." if len(short_name) > 12 else short_name
        
        # TEXTE MINIMAL
        annotation_text = f"<b>{short_name}</b><br>{row['betweenness']:.0f}"
        
        annotations.append(
            dict(
                x=x,
                y=y,
                xref="x",
                yref="y",
                text=annotation_text,
                showarrow=True,
                arrowhead=1,
                arrowwidth=1,
                arrowcolor="#666666",
                bgcolor="rgba(255, 255, 255, 0.95)",
                bordercolor="rgba(0, 0, 0, 0.3)",
                borderwidth=1,
                borderpad=4,
                font=dict(size=10, color="#2c3e50"),
                # Positions stratégiques pour éviter les chevauchements
                ax=0,  # Pas de décalage horizontal
                ay=-40 if y > 0 else 40  # Toujours vers le bas ou le haut
            )
        )

# 6️⃣ Figure épurée
fig_mini = go.Figure(data=[edge_trace] + node_traces)

fig_mini.update_layout(
    title=dict(
        text="🏛️ ARCHITECTES & PASSERELLES<br>"
             "<span style='font-size:11px; color:#666'>"
             "Labels: Top 6 par centralité | Survolez pour les détails"
             "</span>",
        x=0.5,
        xanchor="center",
        y=0.95,
        font=dict(size=14, color=COLOR_PALETTE['text'])
    ),
    width=900,
    height=700,
    showlegend=True,
    hoverlabel=dict(
        bgcolor="rgba(0,0,0,0.8)",
        font_size=10,
        font_color="white"
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    legend=dict(
        title="<b>Rôles</b>",
        bgcolor="rgba(255,255,255,0.9)",
        bordercolor="rgba(0,0,0,0.2)",
        font=dict(size=10),
        x=1.02,
        y=0.5
    ),
    margin=dict(l=20, r=120, t=80, b=20),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    annotations=annotations
)

print("🚀 Graphique épuré - Seulement 6 labels principaux")
fig_mini.show()

Sous-graphe cœur : 35 nœuds, 104 connexions internes
🚀 Graphique épuré - Seulement 6 labels principaux


In [101]:
fig.write_html(
    r"C:\Users\Ashahi\Desktop\Graph\examen\notebooks\graphes_notebook1\graphe_2.html",
    include_plotlyjs='cdn',
    full_html=True
)


### Interpretation

Le réseau montre une coordination claire : les architectes se connectent surtout aux passerelles, qui organisent les flux vers Argyle House. La hiérarchie est : Argyle House (hub central) → passerelles → architectes. Les flux sont redondants, donc les architectes restent connectés au hub même si une passerelle disparaît. Cela montre que le pouvoir est à la fois centralisé autour d’Argyle House et réparti via les passerelles.

 ENTITÉS CAYMAN vs HUB BERMUDA

In [102]:
import networkx as nx
import plotly.graph_objects as go
import numpy as np

# =====================================================
# 1️⃣ Filtrer nœuds Cayman
# =====================================================
cayman_df = df[df['jurisdiction_description'].str.contains('Cayman', na=False)].copy()
cayman_ids = set(cayman_df['id'])

# Inclure les hubs principaux (Top 35 Architectes & Passerelles)
hub_ids = set(top35[top35['typologie'].isin(['Architecte','Passerelle'])]['id'])
G_cayman = G.subgraph(cayman_ids.union(hub_ids)).copy()

print(f"Graphe Cayman ↔ Bermuda : {len(G_cayman.nodes())} nœuds, {len(G_cayman.edges())} arêtes")

# =====================================================
# 2️⃣ Layout
# =====================================================
pos_cayman = nx.spring_layout(G_cayman, k=2, iterations=50, seed=42)

# =====================================================
# 3️⃣ Arêtes
# =====================================================
edge_x, edge_y = [], []
for u, v in G_cayman.edges():
    x0, y0 = pos_cayman[u]
    x1, y1 = pos_cayman[v]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.8,color='rgba(180,180,180,0.4)'),
    hoverinfo='none',
    mode='lines',
    showlegend=False
)

# =====================================================
# 4️⃣ Nœuds avec forme selon Cayman / Bermuda
# =====================================================
node_traces = []
for typ in ['Architecte','Passerelle','Satellite','Intermédiaire secondaire']:
    subset = top35[top35['typologie']==typ]
    subset = subset[subset['id'].isin(G_cayman.nodes())]
    if subset.empty:
        continue
    node_x, node_y, sizes, colors, symbols, hover_texts = [], [], [], [], [], []
    for _, row in subset.iterrows():
        nid = row['id']
        x, y = pos_cayman[nid]
        node_x.append(x)
        node_y.append(y)
        sizes.append(12 + np.log1p(row['degree'])*6)
        colors.append(TYPO_COLORS.get(row['typologie'],"#7f8c8d"))
        # Différencier Cayman et Bermuda par la forme
        symbols.append('square' if 'Cayman' in row['jurisdiction_description'] else 'circle')
        hover_texts.append(
            f"<b>{row['name']}</b><br>Type: {row['node_type']}<br>"
            f"Typologie: {row['typologie']}<br>Pays: {row['jurisdiction_description']}<br>"
            f"Degré: {row['degree']} | Betweenness: {row['betweenness']:.4f}"
        )
    node_traces.append(go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        marker=dict(size=sizes, color=colors, symbol=symbols, line=dict(width=2,color='#000')),
        text=hover_texts,
        hoverinfo='text',
        name=typ
    ))

# =====================================================
# 5️⃣ Figure
# =====================================================
fig_cayman = go.Figure(data=[edge_trace]+node_traces)
fig_cayman.update_layout(
    title="🌍 Interactions Cayman ↔ Bermuda (Top 35)",
    width=1000, height=750, showlegend=True
)

# =====================================================
# 6️⃣ Ajouter annotations pour les top nodes avec fond blanc
# =====================================================
top_nodes = top35[top35['id'].isin(G_cayman.nodes())].nlargest(10, 'betweenness')
annotations = []
for _, row in top_nodes.iterrows():
    nid = row['id']
    x, y = pos_cayman[nid]
    annotations.append(dict(
        x=x,
        y=y,
        xref="x",
        yref="y",
        text=f"{row['name']} ({row['typologie']})",
        showarrow=True,
        arrowhead=2,
        ax=20,
        ay=-20,
        font=dict(size=10, color="#000"),
        bgcolor="white",       # fond blanc
        bordercolor="#000",    # contour noir
        borderwidth=1,
        borderpad=4
    ))

fig_cayman.update_layout(annotations=annotations)

fig_cayman.show()


Graphe Cayman ↔ Bermuda : 40 nœuds, 110 arêtes


In [103]:
fig.write_html(
    r"C:\Users\Ashahi\Desktop\Graph\examen\notebooks\graphes_notebook1\graphe_3.html",
    include_plotlyjs='cdn',
    full_html=True
)


 DYNAMIQUE CLIENT ↔ FOURNISSEUR (Officers ↔ Entities Appleby)

In [104]:
# %% --------------------------------------------------
# 🎯 Graphe Appleby - VERSION ÉQUILIBRÉE
# --------------------------------------------------
import networkx as nx
import plotly.graph_objects as go

# 1️⃣ Détection du nœud Appleby principal
appleby_nodes_in_G = [
    n for n in G.nodes() 
    if 'name' in G.nodes[n] and G.nodes[n]['name'] and 'Appleby' in str(G.nodes[n]['name'])
]

focus_node = max(appleby_nodes_in_G, key=lambda n: G.degree[n])
focus_name = G.nodes[focus_node].get('name', focus_node)

# 2️⃣ Sélection des entités connectées
node_type_attr = 'node_type'
direct_entities = [
    n for n in G.neighbors(focus_node)
    if G.nodes[n].get(node_type_attr, '').lower() == 'entity'
]

# Prendre les 4 entités les plus connectées
direct_entities = sorted(direct_entities, key=lambda n: G.degree[n], reverse=True)[:4]

# 3️⃣ Officers liés aux entités
officers_linked = []
for ent in direct_entities:
    officers = [
        n for n in G.neighbors(ent)
        if G.nodes[n].get(node_type_attr, '').lower() == 'officer'
    ][:2]  # 2 officers max par entité
    officers_linked.extend(officers)

# 4️⃣ Sous-graphe équilibré
subset_nodes = {focus_node} | set(direct_entities) | set(officers_linked)
H_reduced = G.subgraph(subset_nodes).copy()

print(f"📊 Graphe Appleby: {len(H_reduced.nodes())} nœuds, {len(H_reduced.edges())} connexions")

# 5️⃣ Layout bien espacé
pos = nx.spring_layout(H_reduced, k=2, iterations=100, seed=42)

# 6️⃣ Arêtes visibles mais discrètes
edge_x, edge_y = [], []
for src, tgt in H_reduced.edges():
    x0, y0 = pos[src]
    x1, y1 = pos[tgt]
    edge_x += [x0, x1, None]
    edge_y += [y0, y1, None]

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color='rgba(120,120,120,0.4)'),
    hoverinfo='none',
    mode='lines'
)

# 7️⃣ Nœuds avec bonnes informations
traces = []
annotations = []

categories = [
    ("Appleby", [focus_node], "#E63946", 28, "diamond"),
    ("Entités", direct_entities, "#457B9D", 22, "square"),
    ("Officers", officers_linked, "#2A9D8F", 18, "circle")
]

for label, nodes_list, color, size, symbol in categories:
    if not nodes_list:
        continue
    x, y, hover_texts = [], [], []
    
    for n in nodes_list:
        nx_pos, ny_pos = pos[n]
        x.append(nx_pos)
        y.append(ny_pos)
        
        n_name = H_reduced.nodes[n].get('name', str(n))
        n_degree = H_reduced.degree[n]
        
        # Texte de survol informatif
        hover_texts.append(
            f"<b>{n_name}</b><br>"
            f"Type: {label}<br>"
            f"Connexions ici: {n_degree}<br>"
            f"Connexions globales: {G.degree[n]}"
        )
    
    traces.append(go.Scatter(
        x=x, y=y,
        mode='markers',
        marker=dict(
            size=size, 
            color=color, 
            symbol=symbol,
            line=dict(width=2, color='black')
        ),
        text=hover_texts,
        hoverinfo='text',
        name=f"{label} ({len(nodes_list)})"
    ))

# 8️⃣ Annotations pour les nœuds IMPORTANTS seulement
important_nodes = [focus_node] + direct_entities[:3]  # Appleby + 3 entités principales

for i, n in enumerate(important_nodes):
    if n in pos:
        x, y = pos[n]
        n_name = H_reduced.nodes[n].get('name', str(n))
        
        # Nom court mais significatif
        if n == focus_node:
            short_name = "Appleby Services"
        else:
            short_name = n_name.split(';')[0] if ';' in n_name else n_name
            short_name = short_name[:15] + "..." if len(short_name) > 15 else short_name
        
        annotations.append(dict(
            x=x, y=y, xref="x", yref="y",
            text=f"<b>{short_name}</b>",
            showarrow=True,
            arrowhead=2, 
            arrowwidth=1.5, 
            arrowcolor="#333",
            bgcolor="rgba(255,255,255,0.95)",
            bordercolor="rgba(0,0,0,0.3)", 
            borderwidth=1, 
            borderpad=8,
            font=dict(size=11, color="#2c3e50", family="Arial"),
            ax=0, 
            ay=-35 if y > 0 else 35  # Flèches vers l'extérieur
        ))

# 9️⃣ Figure ÉQUILIBRÉE
fig = go.Figure(data=[edge_trace] + traces)

fig.update_layout(
    title=dict(
        text=f"🏢 RÉSEAU {focus_name.split()[0].upper()}<br>"
             "<span style='font-size:12px; color:#666'>"
             "4 entités principales + leurs officers | Survolez pour les détails"
             "</span>",
        x=0.5, 
        xanchor="center",
        font=dict(size=16, color="#2c3e50")
    ),
    showlegend=True,
    legend=dict(
        title="<b>Types de nœuds</b>",
        bgcolor="rgba(255,255,255,0.9)",
        bordercolor="rgba(0,0,0,0.2)",
        font=dict(size=11),
        x=1.02,
        y=0.5
    ),
    hovermode='closest',
    margin=dict(b=30, l=30, r=150, t=80),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    plot_bgcolor="white",
    width=1000,
    height=700,
    annotations=annotations
)

print("🚀 Version équilibrée : Assez d'infos mais restant lisible")
print(f"   • {len(H_reduced.nodes())} nœuds au total")
print(f"   • 4 annotations principales")
print(f"   • Survolez pour tous les détails")
fig.show()

# 🔟 EXPORT
html_filename = "reseau_appleby_equilibre.html"
fig.write_html(html_filename, include_plotlyjs=True, auto_open=False)
print(f"✅ Exporté: {html_filename}")

📊 Graphe Appleby: 9 nœuds, 13 connexions
🚀 Version équilibrée : Assez d'infos mais restant lisible
   • 9 nœuds au total
   • 4 annotations principales
   • Survolez pour tous les détails


✅ Exporté: reseau_appleby_equilibre.html


In [105]:
fig.write_html(
    r"C:\Users\Ashahi\Desktop\Graph\examen\notebooks\graphes_notebook1\graphe_4.html",
    include_plotlyjs='cdn',
    full_html=True
)
